In [6]:
!python -V

Python 3.11.5


#### Q1

In [7]:
import random

known_weather_data = {
    'berlin': 20.0
}

In [8]:
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    temp = random.uniform(-5, 35)
    known_weather_data[city] = temp
    return temp

In [9]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Fetch current weather of the mentioned location",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name only, e.g. Mumbai"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

#### Q2

In [10]:
def set_weather(city: str, temp: float) -> None:
    """
    Sets or updates the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [11]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Update or add weather information for a given city in the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name for which the temperature data should be set, e.g. Berlin"
            },
            "temp": {
                "type": "number",
                "description": "Temperature in Celsius for the specified city"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [56]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-09-04 02:44:59--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py.1’

chat_assistant.py.1 100%[===================>]   3.41K  --.-KB/s    in 0s      

2025-09-04 02:45:00 (30.4 MB/s) - ‘chat_assistant.py.1’ saved [3495/3495]



In [12]:
known_weather_data

{'berlin': 20.0}

In [14]:
from openai import OpenAI
import os
import chat_assistant

# Create OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#  Initialize Tools and register
tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)
tools.add_tool(set_weather, set_weather_tool)

# Chat interface
chat_interface = chat_assistant.ChatInterface()

# Developer prompt
developer_prompt = """You are a helpful assistant. Use the get_weather tool when asked about weather.
You can fetch current weather of a city using get_weather tool and update data using set_weather tool.
If the city name is not provided or only the country or state/province name is provided, you should ask
the user the name of the city for which weather information is required. When you update the weather,
respond naturally to confirm the change."""

# Create Chat Assistant
assistant = chat_assistant.ChatAssistant(tools, developer_prompt, chat_interface, client)

# Run
assistant.run()

You: What's the weather like in Germany?


You: Berlin


You: What about Munich?


You: And Norway?


You: Oslo


...

You: Thanks! So how is the weather at Mumbai?


You: ok. What is Delhi's temperature?


You: Ah great! And what is that of Cairo?


You: Stop


Chat ended.


In [15]:
known_weather_data

{'berlin': 20.0,
 'germany': 34.28511139356317,
 'munich': 26.03104392222005,
 'oslo': 9.883092804555456,
 'delhi': 27,
 'cairo': 25,
 'mumbai': 32.303140887202474}

#### Q3

In [27]:
import json
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        # Convert Tool objects to dicts
        tools_dict = [tool.__dict__ for tool in tools]
        # Print as pretty JSON
        print(json.dumps(tools_dict, indent=4))

await main()

[
    {
        "name": "get_weather",
        "title": null,
        "description": "Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.",
        "inputSchema": {
            "properties": {
                "city": {
                    "title": "City",
                    "type": "string"
                }
            },
            "required": [
                "city"
            ],
            "type": "object"
        },
        "outputSchema": {
            "properties": {
                "result": {
                    "title": "Result",
                    "type": "number"
                }
            },
            "required": [
                "result"
            ],
            "title": "_WrappedResult",
            "type": "object",
            "x-fastmcp-wrap-result": true
        },
        "annotations": null,
        "me

In [16]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/3efe99f5a603bd90439eb5301e30b84ba0296db1/cohorts/2025/0a-agents/mcp_client.py

--2025-09-07 06:47:23--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/3efe99f5a603bd90439eb5301e30b84ba0296db1/cohorts/2025/0a-agents/mcp_client.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9117 (8.9K) [text/plain]
Saving to: ‘mcp_client.py’

mcp_client.py       100%[===================>]   8.90K  --.-KB/s    in 0.001s  

2025-09-07 06:47:23 (17.0 MB/s) - ‘mcp_client.py’ saved [9117/9117]

